<a href="https://colab.research.google.com/github/happiluv/Basic-Model/blob/main/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df =  pd.read_csv('NFA 2019 public_data.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(df)

       country  year  country_code        record     crop_land  grazing_land  \
0      Armenia  1992             1    AreaPerCap  1.402924e-01  1.995463e-01   
1      Armenia  1992             1     AreaTotHA  4.830000e+05  6.870000e+05   
2      Armenia  1992             1  BiocapPerCap  1.598044e-01  1.352610e-01   
3      Armenia  1992             1  BiocapTotGHA  5.501762e+05  4.656780e+05   
4      Armenia  1992             1  EFConsPerCap  3.875102e-01  1.894622e-01   
...        ...   ...           ...           ...           ...           ...   
72181    World  2016          5001  BiocapTotGHA  3.984702e+09  1.504757e+09   
72182    World  2016          5001  EFConsPerCap  5.336445e-01  1.402092e-01   
72183    World  2016          5001  EFConsTotGHA  3.984702e+09  1.046937e+09   
72184    World  2016          5001  EFProdPerCap  5.336445e-01  1.402092e-01   
72185    World  2016          5001  EFProdTotGHA  3.984702e+09  1.046937e+09   

        forest_land  fishing_ground  bu

In [ ]:
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [ ]:
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [ ]:
df['QScore'].describe()

count     72185
unique        5
top          3A
freq      51481
Name: QScore, dtype: object

In [ ]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df['QScore']=df['QScore'].replace(['A'],'2A')

In [ ]:
df.QScore.value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [ ]:
df_2A= df[df.QScore=='2A']
df_3A = df[df.QScore =='3A'].sample(350)

In [ ]:
data_df= df_2A.append(df_3A)

In [ ]:
import sklearn.utils
data_df=sklearn.utils.shuffle(data_df)
data_df=data_df.reset_index(drop=True)
data_df.shape

(574, 12)

In [ ]:
data_df=data_df.drop(columns= ['country_code', 'country', 'year'])
x = data_df.drop(columns='QScore')
y= data_df['QScore'] 

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.3,random_state=0)
y_train.value_counts()

3A    239
2A    162
Name: QScore, dtype: int64

In [ ]:
#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record=encoder.fit_transform(x_train.record)
x_test.record =encoder.fit_transform(x_test.record)

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote= SMOTE(random_state=1)
x_train_balanced,y_balanced = smote.fit_resample(x_train,y_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df=pd.DataFrame(normalised_train_df,columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record']=x_train_balanced['record']


In [ ]:
x_test=x_test.reset_index(drop=True)
normalised_test_df=scaler.transform(x_test.drop(columns=['record']))
normalised_test_df =pd.DataFrame(normalised_test_df,columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record']=x_train_balanced['record']

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df,y_balanced)

LogisticRegression()

In [ ]:
from sklearn.metrics import recall_score,accuracy_score,precision_score,f1_score,confusion_matrix
new_predictions =log_reg.predict(normalised_test_df)
cnf_mat=confusion_matrix(y_true=y_test,y_pred=new_predictions,labels=['2A','3A'])
cnf_mat

array([[48, 14],
       [92, 19]])

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg,normalised_train_df,y_balanced,cv=5,scoring='f1_macro')
scores

array([0.54184612, 0.39478261, 0.41700405, 0.48924731, 0.50328179])

In [ ]:
#accuracy
accuracy=accuracy_score(y_true=y_test,y_pred=new_predictions)
print('Accuracy:{}'.format(round(accuracy*100),2))

Accuracy:39


In [ ]:
#precision
precision=precision_score(y_true=y_test,y_pred=new_predictions,pos_label='2A')
print('precision:{}'.format(round(precision*100),2))


precision:34


In [ ]:
#recall
recall=recall_score(y_true=y_test,y_pred=new_predictions,pos_label='2A')
print('recall:{}'.format(round(recall*100),2))


recall:77


In [ ]:
#f1
f1=f1_score(y_true=y_test,y_pred=new_predictions,pos_label='2A')
print('f1:{}'.format(round(f1*100),2))

f1:48


In [ ]:
#K_FOLD
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores=[]



In [ ]:
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
       x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
       y_train, y_test = y_balanced[train_index], y_balanced[test_index]
       model = LogisticRegression().fit(x_train, y_train)
 #save result to list
f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test),pos_label = '2A')*100)
f1_scores

[0.0, 0.0]

In [ ]:
#stratified kfold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
model = LogisticRegression().fit(x_train, y_train)
#save result to list\n",
f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test),pos_label = '2A')*100)
f1_scores

[68.8]

In [ ]:
#LeaveOneOut
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
average_score
   

52.09205020920502

In [ ]:
#Decision Tree Classifier
#Tree-Based Methods and The Support Vector Machine
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)
   

DecisionTreeClassifier()